In [1]:
import sqlite3 as sql
import pandas  as pd
import numpy as np
import sys
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor
from sklearn import pipeline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import learning_curve, GridSearchCV
from numpy import linalg as LA
import cPickle

In [2]:
def inverse_mutation(sequence,position,current):
    sequence = str(sequence)[:int(position)]+str(current)+str(sequence)[int(position)+1:]
    return sequence
def read_data(file_name):
    conn = sql.connect(file_name)
    cursor = conn.cursor()
    types = [str,str,int,str,float,float,float]
    names = ["sequence","current","position","mutation","ddG","PH","Temp"]
    results = {names[i]:[] for i in range(len(names))}
    for row in cursor.execute("SELECT * FROM dataset"):
        row = list(row)
        row[2] = str(int(row[2])-1)
        for i in range(len(names)):
            results[names[i]].append(types[i](row[i]))
    result = pd.DataFrame(results)
    conn.close()
    return result

In [3]:
def symmetry_add(data_set):
    inversed = data_set.copy()
    inversed.mutation,inversed.current = inversed.current.apply(lambda x:x),inversed.mutation.apply(lambda x:x)
    inversed['ddG'] = inversed.ddG.apply(lambda x: -x)
    inversed['sequence'] = inversed.apply(lambda x: inverse_mutation(x['sequence'],x['position'],x['current']),axis =1)
    return inversed.merge(data_set,how='outer')

In [4]:
def over_sampling(data_set):
    number_plus = len(data_set[data_set.ddG > 0])
    number_minus = len(data_set[data_set.ddG < 0])
    multiply = int(number_minus/number_plus)
    add = number_minus-number_plus*multiply
    plus = data_set[data_set.ddG > 0].copy()
    result = {y:map(lambda x: x[0],plus[[y]].values)*multiply+map(lambda x: x[0],plus[[y]].values)[:add] for y in map(str,plus.columns)}
    result = pd.DataFrame(result)
    all_else = data_set[data_set.ddG <=0].copy()
    return result.merge(all_else,how='outer')

In [5]:
def neighbour(sequence,position,shift):
    if (position + shift > len(sequence)-1 or position + shift < 0):
        return "0"
    else:
        return sequence[position+shift]

In [6]:
def test_true_false(label, true_label):
    count_true = 0
    count_all = 0
    for i in range(len(label)):
        count_all += 1
        if (np.sign(true_label[i]) == np.sign(label[i])):
            count_true += 1
    return float(count_true)/count_all

In [7]:
def test_symmetry(estimator,test_data,test_labels):
    columns = map(str,test_data.columns)
    inversed_test_data = test_data.copy()
    inversed_test_data.current,inversed_test_data.mutation = inversed_test_data.mutation.apply(lambda x: x),inversed_test_data.current.apply(lambda x: x)
    inversed_labels = estimator.predict(inversed_test_data)
    test_labels = np.array(test_labels)
    inversed_labels = np.array(inversed_labels)
    vector = test_labels+inversed_labels
    return np.sum(map(abs,vector))/len(vector)

In [15]:
def calculate():
    #read dataset
    raw_data = read_data("iwdb.sqlite")
    #edit new features
    number_neighbours = 5
    for i in range(1,number_neighbours+1):
        raw_data['Left'+str(i)] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],-(i)),axis=1)
        raw_data['Right'+str(i)] = raw_data.apply(lambda row: neighbour(row['sequence'],row['position'],(i)),axis=1)
    #shuffle data
    raw_data = raw_data.sample(frac=1).reset_index(drop=True)
    #change symbols in features to int values
    inputting = 'A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 0'
    symbols = inputting.split(' ')
    numerics = [i for i in range(len(symbols))]
    categorical = ['current','mutation']+['Left'+str(i) for i in range(1,number_neighbours+1)]+['Right'+str(i) for i in range(1,number_neighbours+1)]
    for item in categorical:
        raw_data[item] = raw_data[item].apply(lambda x: numerics[symbols.index(x)])
    #make train and test data
    train_size = 0.7
    size = int(len(raw_data)*train_size)
    train_data = raw_data.iloc[:size]
    test_data = raw_data.iloc[size:]
    train_data = symmetry_add(train_data)
    #train_data = over_sampling(train_data)
    train_labels = train_data['ddG'].values
    train_data = train_data.drop(['ddG','sequence'],axis=1)
    test_labels = test_data['ddG'].values
    test_data = test_data.drop(['ddG','sequence'],axis=1)
    #make foundation for pipeline
    binary_data_columns = ['holiday', 'workingday']
    binary_data_indices = np.array([(column in binary_data_columns) for column in train_data.columns], dtype = bool)
    categorical_data_indices = np.array([(column in categorical) for column in train_data.columns], dtype = bool)
    numeric_data_columns = ['Temp', 'PH', 'position']
    numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)
    #creat regressor
    regr = RandomForestRegressor(random_state = 0, max_depth = 20, n_estimators = 500)
    #regr = AdaBoostRegressor(random_state=0, n_estimators=100)
    estimator = pipeline.Pipeline(steps = [       
        ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
                #binary
                ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
                #numeric
                ('numeric_variables_processing', pipeline.Pipeline(steps = [
                     ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices]))
                            ])),
        
                #categorical
                ('categorical_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices]))            
                            ])),
            ])),
        ('model_fitting', regr)
        ]
    )
    # make prediction
    estimator.fit(train_data,train_labels)
    predictions = estimator.predict(test_data)
    #save
    #with open('my_dumped_classifier.pkl', 'wb') as fid:
    #    cPickle.dump(regr, fid)   
    #print parameters
    text ="mean absolute error: " + str(metrics.mean_absolute_error(test_labels,predictions))+"\naccuracy: " + str(test_true_false(predictions,test_labels))+"\nmean absolute error symmetry: "+str(test_symmetry(estimator,test_data,test_labels))
    print(text)
    return estimator
    #test_load(test_data,test_labels,binary_data_indices,numeric_data_indices,categorical_data)

In [18]:
calculate()

mean absolute error: 1.073951644891202
accuracy: 0.71686746988
mean absolute error symmetry: 1.0709103489394662


Pipeline(memory=None,
     steps=[('feature_processing', FeatureUnion(n_jobs=None,
       transformer_list=[('binary_variables_processing', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000000000CF63898>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=...imators=500, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [10]:
#with open('weights.pkl', 'wb') as fid:
#    cPickle.dump(regr, fid)

In [11]:
# load it again
def test_load(binary_data_indices,numeric_data_indices,categorical_data_indices):
    with open('weights.pkl', 'rb') as fid:
        gnb_loaded = cPickle.load(fid)
    estimator = pipeline.Pipeline(steps = [       
            ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
                    #binary
                    ('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                    
                    #numeric
                    ('numeric_variables_processing', pipeline.Pipeline(steps = [
                         ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices]))
                                ])),
        
                    #categorical
                    ('categorical_variables_processing', pipeline.Pipeline(steps = [
                        ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices]))            
                                ])),
                ])),
            ('model_fitting', gnb_loaded)
            ]
        )
    return estimator

In [19]:
def prediction(sequence,current,position,mutation,ph = 8.0,temp=37.):
    number_neighbours = 5
    columns = ['sequence','current','position','mutation','PH','Temp']
    data = [sequence,current,position,mutation,ph,temp]
    test_data = {columns[i]:[data[i]] for i in range(len(columns))}
    test_data = pd.DataFrame(test_data)
    for i in range(1,number_neighbours+1):
        test_data['Left'+str(i)] = test_data.apply(lambda row: neighbour(row['sequence'],row['position'],-(i)),axis=1)
        test_data['Right'+str(i)] = test_data.apply(lambda row: neighbour(row['sequence'],row['position'],(i)),axis=1)
    test_data = test_data.drop(['sequence'],axis=1)
    categorical = ['current','mutation']+['Left'+str(i) for i in range(1,number_neighbours+1)]+['Right'+str(i) for i in range(1,number_neighbours+1)]
    binary_data_columns = ['g']
    binary_data_indices = np.array([(column in binary_data_columns) for column in test_data.columns], dtype = bool)
    categorical_data_indices = np.array([(column in categorical) for column in test_data.columns], dtype = bool)
    numeric_data_columns = ['Temp', 'PH', 'position']
    numeric_data_indices = np.array([(column in numeric_data_columns) for column in test_data.columns], dtype = bool)
    inputting = 'A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 0'
    symbols = inputting.split(' ')
    numerics = [i for i in range(len(symbols))]
    for item in categorical:
        test_data[item] = test_data[item].apply(lambda x: numerics[symbols.index(x)])
    estimator = calculate()
    result = estimator.predict(test_data)
    return result
#print("ddG = "+ str(prediction("A","C",5,"G")))